## Objectives of the practical work

The objective is to get hands on experience on the fundamental elements of neural networks:
 
 - perceptron architecture (linear regression)
 - loss function
 - empirical loss
 - gradient descent

For this we will implement from scratch the data-structure and algorithms to train a perceptron. Note that slides related to the perceptron and neural networks in general are available on [moodle](https://moodle.insa-toulouse.fr/course/view.php?id=1790).

## Dataset

The objective of the regression is the prediction of the hydrodynamic performance of sailing yachts from dimensions and velocity.
The **inputs** are linked to dimension and hydrodynamics characteristics:
1. Longitudinal position of the center of buoyancy
(*flottabilité*), adimensional.
2. Prismatic coefficient, adimensional.
3. Length-displacement ratio, adimensional.
4. Beam -draught ratio ((*tiran d’eau*), adimensional.
5. Length-beam ratio, adimensional.
6. Froude number, adimensional

**Target value/predicted value (Output)** = Residuary resistance per unit weight of
displacement, adimensional

In [1]:
# Import some useful libraries and functions

import numpy as np
import pandas

def print_stats(dataset):
    """Print statistics of a dataset"""
    print(pandas.DataFrame(dataset).describe())


In [2]:
# Download the data set and place in the current folder (works on linux only)
filename = 'yacht_hydrodynamics.data'

import os.path
import requests

if not os.path.exists(filename):
    print("Downloading dataset...")
    r = requests.get('https://arbimo.github.io/tp-supervised-learning/tp1/' + filename)
    open(filename , 'wb').write(r.content)
    
print('Dataset available')


Dataset available


### Explore the dataset

- how many examples are there in the dataset? 308 examples.(Each line represents an example, so the total number of lines corresponds to the number of examples in the dataset.)
- how many features for each example? 7 features. (Each line consists of several values separated by spaces. The number of values in a line corresponds to the number of features for that example.)
- what is the ground truth of the 10th example : 1.83. (The last value in the 10thline represents the ground truth for the 10th example.)

In [3]:
# load the dataset and slip between inputs (X) and ground truth (Y)
dataset = np.genfromtxt("yacht_hydrodynamics.data", delimiter='')
X = dataset[:, :-1] # examples features -  all rows with all elements in rows except last one
Y = dataset[:, -1]  # ground truth - last element in all rows
print("number of examples = " + str(len(X)))
print("number of features = " + str(len(Y)))
print("ground truth of the 10th example = " + str(Y[10]))
print("\n")

# Print the first 5 examples
for i in range(0,5):
    print ("example n°"+str(i+1)+" : ")
    print(f"f({X[i]}) = {Y[i]}")
    

number of examples = 308
number of features = 308
ground truth of the 10th example = 32.46


example n°1 : 
f([-5.    0.6   4.78  4.24  3.15  0.35]) = 8.62
example n°2 : 
f([-5.     0.565  4.77   3.99   3.15   0.15 ]) = 0.18
example n°3 : 
f([-2.3    0.565  4.78   5.35   2.76   0.15 ]) = 0.29
example n°4 : 
f([-5.     0.6    4.78   4.24   3.15   0.325]) = 6.2
example n°5 : 
f([0.    0.53  4.78  3.75  3.15  0.175]) = 0.59


The following command adds a column to the inputs.

- what is in the value added this column?
np.ones((len(X))) : array of ones with a length equal to the number of rows in X
numpy.insert(Input array, index of insertion= first column, values inserted in the array= 1s, Axis along which to insert values= vertical)
- why are we doing this?
adding a column of ones at the beginning of the input array helps the model account for any bias or offset in the data, allowing it to make more accurate predictions.
The columns are labeled as 0, 1, 2, 3, 4, 5, and 6.
- count = Count number of non-NA/null observations= umber of data points available for each column
- max = Maximum of the values in the column.
- min = Minimum of the values in the column.
- mean =  average value of each column.
- std = Standard deviation of the observation =  measures the dispersion or variability of the data.
- 25%: The 25th percentile value = first quartile = represents the value below which 25% of the data falls.
- 50%: The 50th percentile value = median = represents the value below which 50% of the data falls.
- 75%: The 75th percentile value = third quartile = represents the value below which 75% of the data falls.

In [4]:
X = np.insert(X, 0, np.ones((len(X))), axis= 1)
print_stats(X)


           0           1           2           3           4           5  \
count  308.0  308.000000  308.000000  308.000000  308.000000  308.000000   
mean     1.0   -2.381818    0.564136    4.788636    3.936818    3.206818   
std      0.0    1.513219    0.023290    0.253057    0.548193    0.247998   
min      1.0   -5.000000    0.530000    4.340000    2.810000    2.730000   
25%      1.0   -2.400000    0.546000    4.770000    3.750000    3.150000   
50%      1.0   -2.300000    0.565000    4.780000    3.955000    3.150000   
75%      1.0   -2.300000    0.574000    5.100000    4.170000    3.510000   
max      1.0    0.000000    0.600000    5.140000    5.350000    3.640000   

                6  
count  308.000000  
mean     0.287500  
std      0.100942  
min      0.125000  
25%      0.200000  
50%      0.287500  
75%      0.375000  
max      0.450000  


## Creating the perceptron

![Perceptron for regression](https://arbimo.github.io/tp-supervised-learning/2223-ae/tp1/perceptron-regression.png)

We now want to define a perceptron, that is, a function of the form: 

$h_w(x) = w_0 + w_1 \times x_1 + \dots + w_n \times x_n$

- Complete the code snippet below to:
  - create the vector of weight `w`, initialize to arbitrary values (we suggest 0)
  - implement the `h` function that evaluate an example based on the vector of weights
  - check if this works on a few examples

In [5]:
w =np.ones(X.shape[1]) # initialization vector weight = w
# implementation of h
def h(w, x):
    result = 0
    for i in range(0,len(w)):
        #evaluate example i based on w 
        result = result + w[i] * x[i]
        
    return result

# print the ground truth and the evaluation of h_w on the first example

print("Evaluation :",h(w,X[1]))
print("Ground truth :",Y[1])

Evaluation : 8.625
Ground truth : 0.18


## Loss function

Complete the definiton of the loss function below such that, for a **single** example `x` with ground truth `y`, it returns the $L_2$ loss of $h_w$ on `x`.

In [6]:
import math
def loss(w, x, y):
    return math.pow(y-h(w,x),2) 


print("L0 : ", loss(w,X[0],Y[0]))
print("L1 : ", loss(w,X[1],Y[1]))

L0 :  0.2500000000000018
L1 :  71.318025


## Empirical loss

Complete the function below to compute the empirical loss of $h_w$ on a **set** of examples $X$ with associated ground truths $Y$.

In [7]:
def emp_loss(w, X, Y):
    aux=0
    for i in range(1,len(X)):
        aux+=loss(w,X[i],Y[i])/len(X)
    return aux

print("Emp_Loss = " + str(emp_loss(w,X,Y)))

Emp_Loss = 229.62787322727277


## Gradient update

A gradient update is of the form: $w \gets w + dw$

- Complete the function below so that it computes the $dw$ term (the 'update') based on a set of examples `(X, Y)` the step (`alpha`)

(you can look at slide 32 of the ANN lecture slides for an example)

In [8]:
def compute_update(w, X, Y, alpha):
    for i in range(len(w)):
        sum=0
        for j in range(len(X)):
            sum+=(Y[j]-h(w,X[j]))*X[j][i]
 
        w[i] += sum*alpha
    return w
    

z=compute_update(w, X, Y, alpha = 10e-7)
print(z)

[0.99972073 1.00009802 0.99983828 0.99861171 0.99878316 0.99909963
 1.00029849]


## Gradient descent

Now implement the gradient descent algorithm that will:

- repeatedly apply an update the weights 
- stops when a max number of iterations is reached (do not consider early stopping for now)
- returns the final vector of weights

In [14]:
def descent(w_init, X, Y, alpha, max_iter):
    tab_loss=np.zeros(max_iter)
    aux=0
    w_init=np.zeros(7)
    for i in range (max_iter):
        w_init=w_init+compute_update(w_init,X,Y,alpha)
        tab_loss[i]=emp_loss(w_init,X,Y)
    return tab_loss,w_init
(tab_loss, w)=descent(w,X,Y,10e-7,50)
print("Tab_loss : " + str(tab_loss) + " \n w = " + str(w))

Tab_loss : [3.31659742e+02 3.17968925e+02 2.93948537e+02 2.57901527e+02
 2.30195285e+02 3.42443761e+02 1.20717586e+03 5.39543107e+03
 2.32425883e+04 9.54704004e+04 3.80964159e+05 1.49664484e+06
 5.83204579e+06 2.26311524e+07 8.76321742e+07 3.38958695e+08
 1.31035408e+09 5.06416347e+09 1.95688030e+10 7.56117008e+10
 2.92144411e+11 1.12875045e+12 4.36108065e+12 1.68495452e+13
 6.51000290e+13 2.51520650e+14 9.71775278e+14 3.75455026e+15
 1.45060759e+16 5.60456550e+16 2.16537917e+17 8.36615624e+17
 3.23234720e+18 1.24884936e+19 4.82505344e+19 1.86420738e+20
 7.20255096e+20 2.78277749e+21 1.07515393e+22 4.15396507e+22
 1.60492617e+23 6.20079400e+23 2.39573941e+24 9.25618186e+24
 3.57621986e+25 1.38170895e+26 5.33837362e+26 2.06253530e+27
 7.96881692e+27 3.07883448e+28] 
 w = [ 3.20303344e+12 -6.93626150e+12  1.79544729e+12  1.52521936e+13
  1.23324623e+13  1.01136209e+13  1.76207201e+12]


## Exploitation

You gradient descent is now complete and you can exploit it to train your perceptron.

- Train your perceptron to get a model.
- Visualize the evolution of the loss on the training set. Has it converged?
- Try training for several choices of `alpha` and `max_iter`. What seem like a reasonable choice?
- What is the loss associated with the final model?
- Is the final model the optimal one for a perceptron?


In [10]:
import matplotlib.pyplot as plt
w=np.zeros(7)
loss1,w=descent(w,X,Y,10e-5,9000)
print(w)
plt.plot(loss1)

OverflowError: math range error

In [ ]:
loss1[-1]

In [ ]:
# Code sample that can be used to visualize the difference between the ground truth and the prediction

num_samples_to_plot = 20
plt.plot(Y[0:num_samples_to_plot], 'ro', label='y')
yw = [h(w,x) for x in X]
plt.plot(yw[0:num_samples_to_plot], 'bx', label='$\hat{y}$')
plt.legend()
plt.xlabel("Examples")
plt.ylabel("f(examples)")







# Going further

The following are extensions of the work previously done. If attempting them **do not modify** the code you produced above so that it can be evaluated.

### Improvements to gradient descent

Consider improving the gradient descent with:

 - Stochastic Gradient Descent (SGD), which means selecting a subset of the examples for training
 - Detection of convergence to halt the algorithm before the maximum number of iterations


### Data normalization

Different input features can have different units, and very different ranges.
Within the perceptron computation, these values will be summed together.
While gradient descent is normally able to deal with this (by adapting the weights of the perceptron for each input feature), standardizing the input features usually eases the perceptron training, and can sometimes improve accuracy.




```python
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(copy=True) 
X_normalized = sc.fit_transform(X)
```

Try applying a standard normalization to the input features (make sure that you keep a feature column that is always equal to 1). Is the convergence faster ? Try to quantify this speed-up. What about accuracy ?